In [1]:
import numpy as np
import os

In [38]:
from stacking_simple import apply_U_rho, load_data, labelsToBitstrings, evaluate_classifier_top_k_accuracy
from stacking_three import copy_state, pred_U_state

In [102]:
base_path = '/mnt/c/Users/vwimalaweera/OneDrive - University College London/Project_Files/project-orthogonal_classifier/myriad_data/03052023162859_650236/classifier_U'
np.random.seed(1)

In [103]:
U_paths = [base_path + '/' + p for p in os.listdir(path)]

In [104]:
U_paths

['/mnt/c/Users/vwimalaweera/OneDrive - University College London/Project_Files/project-orthogonal_classifier/myriad_data/03052023162859_650236/classifier_U/step_100.npy']

# Load data

In [105]:
prefix = "data_dropbox/fashion_mnist/"
trainingPredPath = "new_ortho_d_final_vs_training_predictions.npy"
trainingLabelPath = "ortho_d_final_vs_training_predictions_labels.npy"

N = 1000

trainingPred, trainingLabel = load_data(prefix + trainingPredPath,
          prefix + trainingLabelPath,
          N)

# Calculate the accuracies using the Us. 

In [106]:
U = np.load(U_paths[0])

In [107]:
n_copies = int(np.emath.logn(16, U.shape[0]))
n_copies

3

In [108]:
trainStates = np.array([copy_state(s, n_copies) for s in trainingPred])
trainLabelBs = labelsToBitstrings(trainingLabel, 4)

In [109]:
preds = pred_U_state(trainStates, U)

In [110]:
evaluate_classifier_top_k_accuracy(preds, trainingLabel, 1)

0.73

## Get meta information from path

In [136]:
path = U_paths[0]

In [127]:
path

'/mnt/c/Users/vwimalaweera/OneDrive - University College London/Project_Files/project-orthogonal_classifier/myriad_data/03052023162859_650236/classifier_U/step_100.npy'

In [128]:
r = re.compile('(\d{14})_?(\d{6})?')

In [130]:
obj = r.search(path)
obj.group(1)

'03052023162859'

In [131]:
path = '/mnt/c/Users/vwimalaweera/OneDrive - University College London/Project_Files/project-orthogonal_classifier/myriad_data/03052023162859/classifier_U/step_100.npy'

In [132]:
obj = r.search(path)
obj.group(1)

'03052023162859'

In [133]:
obj.group(2)

In [124]:
r = re.compile('step_(\d{2,3}).npy')

In [126]:
obj = r.search(path)
obj.group(1)

'100'

In [134]:
def get_meta_info(path):
    '''
    Get meta information about classifier including the data, job id (if present) and step number. 
    
    Paths should be of the form: 
    - `.../{date}_{job_id}/.../step_{step_no}.npy`
    - `.../{date}/.../step_{step_no}.npy`
    
    Where `date` is a 14 digit timestamp. `job_id` is 6 digits and `step_no` is number between (10, 999)
    '''
    r_date_jobid = re.compile('(\d{14})_?(\d{6})?')
    r_step = re.compile('step_(\d{2,3}).npy')
    
    searchObj = r_date_jobid.search(path)
    date = searchObj.group(1)
    jobid = searchObj.group(2)
    
    searchObj = r_step.search(path)
    step = searchObj.group(1)
    
    return date, jobid, step

In [137]:
path

'/mnt/c/Users/vwimalaweera/OneDrive - University College London/Project_Files/project-orthogonal_classifier/myriad_data/03052023162859_650236/classifier_U/step_100.npy'

In [138]:
date, jobid, step = get_meta_info(path)

In [139]:
print(date)
print(jobid)
print(step)

03052023162859
650236
100


## Get Z Overlaps

In [140]:
from stacking_simple import calculate_ZOverlap, generate_Zi

In [145]:
U = np.load(U_paths[0])

In [143]:
qNo = int(np.log2(trainStates.shape[1]))
label_start = qNo-4

In [146]:
Zls = []
for i in range(qNo - label_start):
    Zi = generate_Zi(qNo, i+1+label_start)
    Zoverlaps = np.real(calculate_ZOverlap(trainStates, U, Zi))
    Zls.append(Zoverlaps)

In [149]:
Zls = np.array(Zls)

In [150]:
Zls.shape

(4, 1000)

## Extracting unitaries from tar.gz files

In [40]:
path = '/mnt/c/Users/vwimalaweera/OneDrive - University College London/Project_Files/project-orthogonal_classifier/myriad_data'
libs = os.listdir(path)

In [41]:
libs

['03052023122620',
 '03052023122620_646288',
 'files_from_job_646287.tar.gz',
 'files_from_job_646288.tar.gz',
 'files_from_job_650236.tar.gz',
 'stacking_three_copy.e646287',
 'stacking_three_copy.e650236',
 'stacking_three_copy.e711727',
 'stacking_three_copy.o646287',
 'stacking_three_copy.o650236',
 'stacking_three_copy.o711727',
 'stacking_three_copy_red.e646288',
 'stacking_three_copy_red.e711729',
 'stacking_three_copy_red.o646288',
 'stacking_three_copy_red.o711729']

In [42]:
import re

In [81]:
r = re.compile("\.tar\.gz$")

In [85]:
tar_files = []
for p in libs:
    if r.search(p):
        print(f'Matched: {p}')
        tar_files.append(path + '/' + p)

Matched: files_from_job_646287.tar.gz
Matched: files_from_job_646288.tar.gz
Matched: files_from_job_650236.tar.gz


In [86]:
tar_files

['/mnt/c/Users/vwimalaweera/OneDrive - University College London/Project_Files/project-orthogonal_classifier/myriad_data/files_from_job_646287.tar.gz',
 '/mnt/c/Users/vwimalaweera/OneDrive - University College London/Project_Files/project-orthogonal_classifier/myriad_data/files_from_job_646288.tar.gz',
 '/mnt/c/Users/vwimalaweera/OneDrive - University College London/Project_Files/project-orthogonal_classifier/myriad_data/files_from_job_650236.tar.gz']

In [87]:
import tarfile

In [100]:
with tarfile.open(tar_files[0], 'r') as tfile:
    #print(tfile.list(verbose=False))
    
    # obj = tfile.extract('tmpdir/job/646287.undefined/tanh_3_copy/03052023122620/classifier_U/step_100.npy')
    obj = tfile.extractfile('tmpdir/job/646287.undefined/tanh_3_copy/03052023122620/run_data.csv')
    
    data = np.loadtxt(obj, delimiter=',')
    obj.close()

In [101]:
data

array([[0.776     , 3.45892107, 0.05      ],
       [0.694     , 3.45892107, 0.05      ],
       [0.672     , 3.17664439, 0.04830918],
       [0.708     , 2.96171115, 0.04672897],
       [0.666     , 3.11310782, 0.04524887],
       [0.702     , 3.14378702, 0.04385965],
       [0.75      , 3.13686443, 0.04255319],
       [0.75      , 3.18939753, 0.04132231],
       [0.726     , 3.2831634 , 0.04016064],
       [0.712     , 3.14651937, 0.0390625 ],
       [0.684     , 3.2289284 , 0.03802281],
       [0.61      , 3.19567755, 0.03703704],
       [0.622     , 2.91432093, 0.03610108],
       [0.716     , 2.95197821, 0.03521127],
       [0.734     , 3.23262837, 0.03436426],
       [0.752     , 3.25317271, 0.03355705],
       [0.746     , 3.22207511, 0.03278689],
       [0.756     , 3.20483737, 0.03205128],
       [0.718     , 3.26375206, 0.03134796],
       [0.734     , 3.1656946 , 0.03067485],
       [0.738     , 3.23975117, 0.03003003],
       [0.706     , 3.20704183, 0.02941176],
       [0.